In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Bidirectional, GRU, Dense, Flatten, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
import os
#print(os.listdir('household-consumption'))
df = pd.read_csv('household_power_consumption.txt', 
                 sep=';', na_values=['nan', '?'])
df['dt'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('dt', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)

In [9]:
df.interpolate(method='linear', inplace=True)  # Interpolate missing values
df.ffill(inplace=True)  # Forward fill missing values
 # Forward fill as a backup

In [10]:
# Feature Engineering
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['year'] = df.index.year

In [11]:
# Define target and features
target = 'Global_active_power'  # Assuming this is the correct column
df[target] = df[target].astype(float)  # Ensure correct dtype
features = [col for col in df.columns if col != target]

In [12]:
# Normalize features
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [13]:
# Create sequences
def create_sequences(df, target, seq_length=48):
    num_samples = len(df) - seq_length
    X = np.zeros((num_samples, seq_length, len(features)))
    y = np.zeros(num_samples)

    df_values = df[features].values  # Convert to NumPy array once for efficiency
    target_values = df[target].values

    for i in range(num_samples):
        X[i] = df_values[i:i+seq_length]
        y[i] = target_values[i+seq_length]

    return X, y

seq_length = 24  # Using a longer time window
X, y = create_sequences(df, target, seq_length)

In [14]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [15]:
# Self-Attention Layer
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(SelfAttention, self).__init__()
        self.W_q = Dense(units)
        self.W_k = Dense(units)
        self.W_v = Dense(units)
        self.W_o = Dense(units)  # Added output projection to ensure shape consistency
        self.layer_norm = LayerNormalization()
        self.units = units

    def call(self, inputs):
        Q = self.W_q(inputs)
        K = self.W_k(inputs)
        V = self.W_v(inputs)
        attention_scores = tf.nn.softmax(tf.matmul(Q, K, transpose_b=True) / tf.sqrt(tf.cast(self.units, tf.float32)))
        attention_output = tf.matmul(attention_scores, V)
        attention_output = self.W_o(attention_output)  # Ensures shape consistency
        return self.layer_norm(attention_output + inputs)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Bidirectional, GRU, Dense, Flatten, Dropout, LeakyReLU, BatchNormalization, LayerNormalization, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define Input Shape (time_steps, features)
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# CNN Branch
cnn = Conv1D(filters=32, kernel_size=2, padding='same', kernel_initializer='he_normal')(input_layer)
cnn = LeakyReLU()(cnn)
cnn = BatchNormalization()(cnn)
cnn = Dropout(0.3)(cnn)

cnn = Conv1D(filters=64, kernel_size=2, padding='same', kernel_initializer='he_normal')(cnn)
cnn = LeakyReLU()(cnn)
cnn = BatchNormalization()(cnn)
cnn = Dropout(0.3)(cnn)
cnn = Flatten()(cnn)

# BiGRU Branch
bigru = Bidirectional(GRU(32, return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01), recurrent_dropout=0.2))(input_layer)
bigru = LayerNormalization()(bigru)
bigru = Dropout(0.3)(bigru)

bigru = Bidirectional(GRU(64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01), recurrent_dropout=0.2))(bigru)
bigru = LayerNormalization()(bigru)
bigru = Dropout(0.3)(bigru)

# Attention Mechanism
attention = Attention()([bigru, bigru])
attention = Flatten()(attention)

# Merge both branches
merged = tf.keras.layers.Concatenate()([cnn, attention])

# Fully connected layers
dense = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(merged)
out = Dense(1, dtype='float32')(dense)  # Ensure float32 output

# Define Model
model = Model(inputs=input_layer, outputs=out)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 10)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 24, 32)       672         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 24, 32)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 24, 64)       8448        input_1[0][0]                    
______________________________________________________________________________________________

In [17]:
# Define Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)

""""# Define callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]
"""
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=256,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)
model.save('model_Trail.h5')

Epoch 1/25
1569/5189 [========>.....................] - ETA: 18:22 - loss: 0.9080

KeyboardInterrupt: 

In [ ]:
# Predictions
y_pred = model.predict(X_test)

In [ ]:
# Compute evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Print results
print(f'MSE: {mse:.4f}')
print(f'R2 Score: {r2:.4f}')
print(f'MAPE: {mape:.2f}%')


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history.get('val_loss', []), label='Validation Loss')  # Use `.get()` to avoid KeyError
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
